# Introduction
This file will be used to format the training set for the different tests

We will begin by clearing folders that have been created by this same script. So we can get a fresh setup

In [ ]:
%reset -f
import os
import shutil
import random
from tqdm import tqdm  # For the progress bar

# Path to the 'dataset' folder
dataset_folder = "dataset"

# List of folders to keep
# folders_to_keep = ["images", "labels"]
folders_to_keep = []

# Iterate through all items in the dataset folder
for item in os.listdir(dataset_folder):
    item_path = os.path.join(dataset_folder, item)
    
    # Check if the item is not in the keep list
    if item not in folders_to_keep:
        # Remove the folder or file
        if os.path.isdir(item_path):
            shutil.rmtree(item_path)  # Remove directories
            print(f"Removed folder: {item_path}")
        else:
            os.remove(item_path)  # Remove files
            print(f"Removed file: {item_path}")



The following code downloads the COCO dataset from github.

In [ ]:
from ultralytics.utils.downloads import download
from pathlib import Path

# Define the dataset folder path
dataset_dir = Path().parent / "dataset"  # Set dataset path relative to the script

# Ensure the dataset folder exists
dataset_dir.mkdir(parents=True, exist_ok=True)

# Download labels
segments = True  # segment or box labels
url = 'https://github.com/ultralytics/assets/releases/download/v0.0.0/'
urls = [url + ('coco2017labels-segments.zip' if segments else 'coco2017labels.zip')]  # labels
download(urls, dir=dataset_dir / 'labels')

# Download images
urls = [
    'http://images.cocodataset.org/zips/train2017.zip',  # 19G, 118k images
    'http://images.cocodataset.org/zips/val2017.zip'  # 1G, 5k images
    # ,'http://images.cocodataset.org/zips/test2017.zip'  # 7G, 41k images (optional)
]
download(urls, dir=dataset_dir / 'images', threads=3)

print(f"Labels saved to {dataset_dir / 'labels'}")
print(f"Images saved to {dataset_dir / 'images'}")


# Image reordering
Copies all the images from ~/datasets/coco into the /dataset folder

In [ ]:
%reset -f
import os
import shutil
from tqdm import tqdm

# Define source directories
coco_train_images = os.path.expanduser("dataset/images/train2017")
coco_val_images = os.path.expanduser("dataset/images/val2017")
coco_train_labels = os.path.expanduser("dataset/labels/coco/labels/train2017")
coco_val_labels = os.path.expanduser("dataset/labels/coco/labels/val2017")

# Define target directories
dataset_images = "dataset/images"
dataset_labels = "dataset/labels"

# Ensure target directories exist
os.makedirs(dataset_images, exist_ok=True)
os.makedirs(dataset_labels, exist_ok=True)

# Helper function to copy files
def copy_files(source_dir, target_dir, file_extension, phase_name):
    """
    Copy files with a specific extension from source_dir to target_dir.

    Args:
        source_dir (str): Source directory path.
        target_dir (str): Target directory path.
        file_extension (str): File extension to filter.
        phase_name (str): Name of the phase (e.g., 'train', 'val') for progress.
    """
    files = [f for f in os.listdir(source_dir) if f.endswith(file_extension)]
    
    with tqdm(total=len(files), desc=f"Copying {phase_name} files", unit="file") as pbar:
        for file_name in files:
            src_path = os.path.relpath(os.path.join(source_dir, file_name))
            dst_path = os.path.relpath(os.path.join(target_dir, file_name))
            shutil.move(src_path, dst_path)
            pbar.update(1)

# Copy files
copy_files(coco_train_images, dataset_images, ".jpg", "Training Images")
copy_files(coco_val_images, dataset_images, ".jpg", "Validation Images")
copy_files(coco_train_labels, dataset_labels, ".txt", "Training Labels")
copy_files(coco_val_labels, dataset_labels, ".txt", "Validation Labels")

print("Files successfully copied to dataset/images and dataset/labels with relative paths.")




Clear unused files that have been downloaded from the COCO dataset

In [ ]:
import os

# Directories to clean
directories = [
    "dataset/images",
    "dataset/labels"
]

def clean_directory(directory, extensions=[".jpg", ".txt"]):
    """
    Removes all files and folders from a directory except those with the specified extensions.
    
    Args:
        directory (str): The directory to clean.
        extensions (list): The file extensions to keep (default: [".jpg", ".txt"]).
    """
    for item in os.listdir(directory):
        item_path = os.path.join(directory, item)
        # Check if the item is a file and doesn't have a desired extension
        if os.path.isfile(item_path) and not any(item.lower().endswith(ext) for ext in extensions):
            os.remove(item_path)  # Remove the file
        # If the item is a folder, remove it
        elif os.path.isdir(item_path):
            for root, dirs, files in os.walk(item_path, topdown=False):
                for name in files:
                    os.remove(os.path.join(root, name))
                for name in dirs:
                    os.rmdir(os.path.join(root, name))
            os.rmdir(item_path)  # Remove the directory itself

# Clean each directory
for directory in directories:
    if os.path.exists(directory):
        clean_directory(directory)
        print(f"Cleaned directory: {directory}")
    else:
        print(f"Directory does not exist: {directory}")

print("Cleaning complete!")


### Reformat dataset to include only vehicles.
The original dataset contains the labels of:

    ["aeroplane", "bicyclebike", "bird", "boat", "bottle", "bus",
    "car", "cat", "chair", "cow", "diningtable", "dog", "horse",
    "motorbike", "person", "pottedplant", "sheep", "sofa", "train", "tvmonitor"]

A new filtered label directory has been created so that only remain the vehicles:

    ["car", "bus", "motorbike", "bicyclebike"]

These new labels are stored in dataset/labels_filtered

In [ ]:
# Directory containing YOLO label .txt files
label_dir = "dataset/labels"  # Replace with your label directory path

# Allowed Class IDs for vehicle-related objects
ALLOWED_CLASSES = {0, 1, 2, 3, 4, 5, 6, 7}  # person, bicycle, car, motorcycle, airplane, bus, train, truck

# Directory to save filtered labels
output_dir = "dataset/labels_filtered"
os.makedirs(output_dir, exist_ok=True)

def filter_labels(label_file):
    """
    Reads a YOLO label file, filters out unwanted classes,
    and writes the remaining labels to a new file.
    """
    input_path = os.path.join(label_dir, label_file)
    output_path = os.path.join(output_dir, label_file)

    with open(input_path, "r") as infile, open(output_path, "w") as outfile:
        for line in infile:
            parts = line.split()
            class_id = int(parts[0])  # Extract class ID
            if class_id in ALLOWED_CLASSES:
                # Write the line if class ID is allowed
                outfile.write(line)

# List all .txt files in the label directory
label_files = [f for f in os.listdir(label_dir) if f.endswith(".txt")]

# Process all .txt files with a progress bar
with tqdm(total=len(label_files), desc="Filtering Labels", unit="file") as pbar:
    for file_name in label_files:
        filter_labels(file_name)
        pbar.update(1)

print(f"Filtered labels saved in: {output_dir}")

### Remove empty labels

In [ ]:
# Define the path to the labels folder
labels_folder = "dataset/labels_filtered"

# List all .txt files in the labels folder
label_files = [f for f in os.listdir(labels_folder) if f.endswith('.txt')]

# Initialize a counter for removed files
removed_count = 0

# Check each label file and remove it if it's empty
for label_file in label_files:
    label_path = os.path.join(labels_folder, label_file)
    if os.path.getsize(label_path) == 0:  # Check if the file size is 0 bytes
        os.remove(label_path)  # Remove the empty file
        removed_count += 1
        # print(f"Removed empty label: {label_file}")

# Output the result
print(f"Total empty labels removed: {removed_count}")



Counting images and filtered_labels

In [ ]:
print(os.getcwd())
# Define the paths to the images and labels folders
images_folder = "dataset/images"
labels_folder = "dataset/labels_filtered"

# List all files in the images and labels folders
image_files = [f for f in os.listdir(images_folder) if f.endswith('.jpg')]
label_files = [f for f in os.listdir(labels_folder) if f.endswith('.txt')]

# Count the total number of images and labels
num_images = len(image_files)
num_labels = len(label_files)

# Check for matching files (base filenames without extensions)
image_basenames = {os.path.splitext(f)[0] for f in image_files}
label_basenames = {os.path.splitext(f)[0] for f in label_files}

# Count matched and unmatched files
matched_files = image_basenames & label_basenames
unmatched_images = image_basenames - label_basenames
unmatched_labels = label_basenames - image_basenames

print(f"Total images: {num_images}")
print(f"Total labels: {num_labels}")
print(f"Matched files: {len(matched_files)}")
print(f"Unmatched images: {len(unmatched_images)}")
print(f"Unmatched labels: {len(unmatched_labels)}")

# Optionally print the unmatched files
if unmatched_images:
    print("Unmatched images (no corresponding label):")
    for img in unmatched_images:
        print(f"  {img}")

if unmatched_labels:
    print("Unmatched labels (no corresponding image):")
    for lbl in unmatched_labels:
        print(f"  {lbl}")


Create the folder of images_filtered with a reduced number of unlabellel images.
The ratio of labelled images and unlabelled images has been set to 50/50. 

In [ ]:
import random

# Paths
images_folder = "dataset/images"
labels_folder = "dataset/labels_filtered"
output_folder = "dataset/images_filtered"

# Ratio of labeled and unlabeled images
r_label = 50
r_unlabel = 100 - r_label

# Ensure output directory exists
os.makedirs(output_folder, exist_ok=True)

# Get all image files and corresponding label files
image_files = [f for f in os.listdir(images_folder) if f.endswith('.jpg')]
label_files = [f for f in os.listdir(labels_folder) if f.endswith('.txt')]

# Get the base filenames (without extensions) for labels
label_basenames = {os.path.splitext(label)[0] for label in label_files}

# Separate labeled and unlabeled images
labeled_images = [img for img in image_files if os.path.splitext(img)[0] in label_basenames]
unlabeled_images = [img for img in image_files if os.path.splitext(img)[0] not in label_basenames]

# Check counts
num_labeled = len(labeled_images)
num_unlabeled_to_select = min(int(num_labeled * r_unlabel / r_label), len(unlabeled_images))

# Randomly select the required number of unlabeled images
selected_unlabeled_images = random.sample(unlabeled_images, num_unlabeled_to_select)

# Combine labeled and selected unlabeled images
images_to_copy = labeled_images + selected_unlabeled_images

# Copy labeled and selected unlabeled images to the output folder with a progress bar
with tqdm(total=len(images_to_copy), desc="Copying Images", unit="file") as pbar:
    for img in images_to_copy:
        src_path = os.path.join(images_folder, img)
        dst_path = os.path.join(output_folder, img)
        shutil.copy(src_path, dst_path)
        pbar.update(1)

# Output results
print(f"Total labeled images: {num_labeled}")
print(f"Total unlabeled images selected: {len(selected_unlabeled_images)}")
print(f"Total images in 'images_filtered': {len(os.listdir(output_folder))}")



# Create Train, Validation and Test image sets
From the image and labels ("dataset/images", "dataset/labels_filtered")
Create the test, validation and test sets.


- Training is stored in ("dataset/train/images", "dataset/train/labels")
- Validation is stored in ("dataset/valid/images", "dataset/valid/labels")
- Test is stored in ("dataset/test/images", "dataset/test/labels")


The following code takes the unfiltered images and create the corresponding training, validation and test set in the following folders:
- dataset/train
- dataset/valid
- dataset/test

In [ ]:
import os
import shutil
import random
from tqdm import tqdm

# Split data into 20% train, 5% validation, and 5% test
train_perc = 0.8
valid_perc = 0.1
test_perc = 0.1

# Define folder paths
images_folder = "dataset/images"  # Folder containing filtered images
labels_folder = "dataset/labels"  # Folder containing filtered labels

train_images_folder = "dataset/train/images"
train_labels_folder = "dataset/train/labels"

valid_images_folder = "dataset/valid/images"
valid_labels_folder = "dataset/valid/labels"

test_images_folder = "dataset/test/images"
test_labels_folder = "dataset/test/labels"

# Create output directories
for folder in [train_images_folder, train_labels_folder,
               valid_images_folder, valid_labels_folder,
               test_images_folder, test_labels_folder]:
    if os.path.exists(folder):
        # Remove the folder and its contents
        shutil.rmtree(folder, ignore_errors=True)
    
    os.makedirs(folder, exist_ok=True)

# Get a list of all images
image_files = sorted(os.listdir(images_folder))

# Create a list of images with and without labels
data = []
for image_file in image_files:
    label_file = os.path.splitext(image_file)[0] + ".txt"
    if os.path.exists(os.path.join(labels_folder, label_file)):
        data.append((image_file, label_file))  # Image has a corresponding label
    else:
        data.append((image_file, None))  # Image has no label (no objects detected)

# Shuffle the data
random.shuffle(data)

# Calculate splits
total_data = len(data)
train_split = int(train_perc * total_data)
valid_split = train_split + int(valid_perc * total_data)
test_split = valid_split + int(test_perc * total_data)

# Allocate data
train_data = data[:train_split]
valid_data = data[train_split:valid_split]
test_data = data[valid_split:test_split]

# Function to copy images and labels with a progress bar
def copy_files(data, dest_images_folder, dest_labels_folder, phase_name):
    with tqdm(total=len(data), desc=f"Copying {phase_name}") as pbar:
        for image_file, label_file in data:
            # Copy the image file
            shutil.copy(os.path.join(images_folder, image_file), os.path.join(dest_images_folder, image_file))
            # Copy the label file if it exists
            if label_file:
                shutil.copy(os.path.join(labels_folder, label_file), os.path.join(dest_labels_folder, label_file))
            # Update progress bar
            pbar.update(1)

# Copy data to respective folders
copy_files(train_data, train_images_folder, train_labels_folder, "Training Data")
copy_files(valid_data, valid_images_folder, valid_labels_folder, "Validation Data")
copy_files(test_data, test_images_folder, test_labels_folder, "Testing Data")

print("Dataset split complete!")
print(f"Training data: {len(train_data)} images")
print(f"Validation data: {len(valid_data)} images")
print(f"Testing data: {len(test_data)} images")


The following code takes the filtered images and create the corresponding training, validation and test set in the following folders:
- dataset/train_filtered
- dataset/valid_filtered
- dataset/test_filtered


In [ ]:
import os
import shutil
import random
from tqdm import tqdm

# Split data into 20% train, 5% validation, and 5% test
train_perc = 0.8
valid_perc = 0.1
test_perc = 0.1

# Define folder paths
images_folder = "dataset/images_filtered"  # Folder containing filtered images
labels_folder = "dataset/labels_filtered"  # Folder containing filtered labels

train_images_folder = "dataset/train_filtered/images"
train_labels_folder = "dataset/train_filtered/labels"

valid_images_folder = "dataset/valid_filtered/images"
valid_labels_folder = "dataset/valid_filtered/labels"

test_images_folder = "dataset/test_filtered/images"
test_labels_folder = "dataset/test_filtered/labels"

# Create output directories
for folder in [train_images_folder, train_labels_folder,
               valid_images_folder, valid_labels_folder,
               test_images_folder, test_labels_folder]:
    if os.path.exists(folder):
        # Remove the folder and its contents
        shutil.rmtree(folder, ignore_errors=True)
    
    os.makedirs(folder, exist_ok=True)

# Get a list of all images
image_files = sorted(os.listdir(images_folder))

# Create a list of images with and without labels
data = []
for image_file in image_files:
    label_file = os.path.splitext(image_file)[0] + ".txt"
    if os.path.exists(os.path.join(labels_folder, label_file)):
        data.append((image_file, label_file))  # Image has a corresponding label
    else:
        data.append((image_file, None))  # Image has no label (no objects detected)

# Shuffle the data
random.shuffle(data)

# Calculate splits
total_data = len(data)
train_split = int(train_perc * total_data)
valid_split = train_split + int(valid_perc * total_data)
test_split = valid_split + int(test_perc * total_data)

# Allocate data
train_data = data[:train_split]
valid_data = data[train_split:valid_split]
test_data = data[valid_split:test_split]

# Function to copy images and labels with a progress bar
def copy_files(data, dest_images_folder, dest_labels_folder, phase_name):
    with tqdm(total=len(data), desc=f"Copying {phase_name}") as pbar:
        for image_file, label_file in data:
            # Copy the image file
            shutil.copy(os.path.join(images_folder, image_file), os.path.join(dest_images_folder, image_file))
            # Copy the label file if it exists
            if label_file:
                shutil.copy(os.path.join(labels_folder, label_file), os.path.join(dest_labels_folder, label_file))
            # Update progress bar
            pbar.update(1)

# Copy data to respective folders
copy_files(train_data, train_images_folder, train_labels_folder, "Training Data")
copy_files(valid_data, valid_images_folder, valid_labels_folder, "Validation Data")
copy_files(test_data, test_images_folder, test_labels_folder, "Testing Data")

print("Dataset split complete!")
print(f"Training data: {len(train_data)} images")
print(f"Validation data: {len(valid_data)} images")
print(f"Testing data: {len(test_data)} images")


In [ ]:
import os
from collections import defaultdict
from tqdm import tqdm

# Directories to process
directories = [
    "dataset/labels",
    "dataset/train/labels",
    "dataset/valid/labels",
    "dataset/test/labels",
    
    "dataset/labels_filtered",
    "dataset/train_filtered/labels",
    "dataset/valid_filtered/labels",
    "dataset/test_filtered/labels",
]

def count_labels_in_directory(directory):
    """
    Counts the number of each class in a directory of YOLO label files with a progress bar.
    
    Args:
        directory (str): Path to the directory containing label files.
        
    Returns:
        dict: A dictionary with class IDs as keys and counts as values.
    """
    class_counts = defaultdict(int)
    
    # Get all .txt files in the directory
    label_files = [f for f in os.listdir(directory) if f.endswith(".txt")]
    
    # Process each file with a progress bar
    with tqdm(total=len(label_files), desc=f"Processing {directory}", unit="file") as pbar:
        for label_file in label_files:
            label_path = os.path.join(directory, label_file)
            with open(label_path, "r") as file:
                for line in file:
                    parts = line.split()
                    class_id = int(parts[0])  # Extract class ID
                    class_counts[class_id] += 1  # Increment the count for the class ID
            pbar.update(1)
    
    return class_counts

# Count labels for each directory
results = {}

for directory in directories:
    if os.path.exists(directory):
        results[directory] = count_labels_in_directory(directory)
    else:
        results[directory] = None  # Directory does not exist

# Display the results
print("\nClass counts per directory:")
for directory, class_counts in results.items():
    print(f"\nDirectory: {directory}")
    if class_counts is not None:
        for class_id, count in sorted(class_counts.items()):
            print(f"  Class {class_id}: {count}")
    else:
        print("  Directory does not exist or contains no labels.")



# Load YOLO Model and Begin Training!!

In [1]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())    
torch.cuda.empty_cache()

2.6.0+cu126
True


In [2]:
import torch

# Check for available devices
device_name = "cuda" if torch.cuda.is_available() else "cpu"
available_devices = torch.cuda.device_count()

print(f"Device in use: {device_name}")
print(f"Available CUDA devices: {available_devices}")

Device in use: cuda
Available CUDA devices: 1


In [3]:
import torch
print(torch.version.cuda)
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(torch.cuda.current_device()))


12.6
True
0
NVIDIA GeForce RTX 3070


In [4]:
from ultralytics import YOLO
!yolo checks

Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\remote_student\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


Ultralytics 8.3.71 🚀 Python-3.11.9 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 3070, 8192MiB)
Setup complete ✅ (24 CPUs, 31.7 GB RAM, 222.1/952.9 GB disk)

OS                  Windows-10-10.0.26100-SP0
Environment         Windows
Python              3.11.9
Install             pip
RAM                 31.70 GB
Disk                222.1/952.9 GB
CPU                 12th Gen Intel Core(TM) i9-12900
CPU count           24
GPU                 NVIDIA GeForce RTX 3070, 8192MiB
GPU count           1
CUDA                12.6

numpy               ✅ 2.1.1<=2.1.1,>=1.23.0
matplotlib          ✅ 3.10.0>=3.3.0
opencv-python       ✅ 4.11.0.86>=4.6.0
pillow              

In [5]:
from ultralytics import YOLO

model16 = YOLO("yolo11n.yaml")

model16.train(
    data="train_filtered.yaml",
    project="./16_scratch_filtered",
    pretrained=False,  
    epochs = 200,
    patience=10, 
    # save_period=10,
    # time=2,
    # cache="ram",
    batch=16,
    exist_ok=True
    )

    100/200      3.17G      1.167      1.139      1.205         38        640: 100%|██████████| 6165/6165 [10:48<00:00,  9.51it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 386/386 [00:36<00:00, 10.64it/s]


                   all      12328      35564      0.728      0.535      0.617      0.433

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    101/200      3.04G      1.162      1.133      1.202         30        640: 100%|██████████| 6165/6165 [10:41<00:00,  9.61it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 386/386 [00:35<00:00, 10.94it/s]


                   all      12328      35564      0.722      0.536      0.617      0.434

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    102/200      3.28G      1.161      1.132      1.202          7        640: 100%|██████████| 6165/6165 [10:42<00:00,  9.59it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 386/386 [00:34<00:00, 11.09it/s]


...

    200/200       2.8G      1.002      0.803      1.095         15        640: 100%|██████████| 6165/6165 [10:30<00:00,  9.77it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 386/386 [00:35<00:00, 10.95it/s]


                   all      12328      35564      0.767      0.576      0.665      0.473

200 epochs completed in 37.730 hours.
Optimizer stripped from 16_scratch_filtered\train\weights\last.pt, 5.5MB
Optimizer stripped from 16_scratch_filtered\train\weights\best.pt, 5.5MB

Validating 16_scratch_filtered\train\weights\best.pt...
Ultralytics 8.3.71  Python-3.11.9 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 3070, 8192MiB)
YOLO11n summary (fused): 238 layers, 2,583,712 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 386/386 [00:35<00:00, 10.91it/s]


                   all      12328      35564      0.765      0.574      0.665      0.474
                person       6695      27034      0.826      0.652      0.762      0.528
               bicycle        359        691      0.751      0.379      0.476      0.291
                   car       1232       4330      0.714      0.498      0.576      0.367
            motorcycle        346        802      0.792      0.584      0.692      0.451
              airplane        312        596      0.812      0.621      0.716       0.52
                   bus        414        639      0.823      0.667      0.763      0.612
                 train        386        487      0.819      0.805      0.856      0.671
                 truck        632        985      0.586      0.389       0.48      0.348
Speed: 0.1ms preprocess, 0.8ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to 16_scratch_filtered\train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 7])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000024017DA4890>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047

In [9]:
from ultralytics import YOLO

model16 = YOLO("yolo11n.yaml")

model16.train(
    data="train_filtered.yaml",
    project="./16_scratch_filtered",
    pretrained=False,  
    epochs = 200,
    patience=10, 
    # save_period=10,
    # time=2,
    # cache="ram",
    batch=16,
    exist_ok=True
    )

Ultralytics 8.3.71  Python-3.11.9 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 3070, 8192MiB)
engine\trainer: task=detect, mode=train, model=yolo11n.yaml, data=train_filtered.yaml, epochs=200, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=./16_scratch_filtered, name=train, exist_ok=True, pretrained=False, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, sh

NotImplementedError: Could not run 'torchvision::nms' with arguments from the 'CUDA' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'torchvision::nms' is only available for these backends: [CPU, Meta, QuantizedCPU, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradXLA, AutogradMPS, AutogradXPU, AutogradHPU, AutogradLazy, AutogradMTIA, AutogradMeta, Tracer, AutocastCPU, AutocastXPU, AutocastMPS, AutocastCUDA, FuncTorchBatched, BatchedNestedTensor, FuncTorchVmapMode, Batched, VmapMode, FuncTorchGradWrapper, PythonTLSSnapshot, FuncTorchDynamicLayerFrontMode, PreDispatch, PythonDispatcher].

CPU: registered at C:\actions-runner\_work\vis

In [ ]:
from ultralytics import YOLO
model16 = YOLO("16_scratch_filtered/train/weights/last.pt")
model16.train(resume=True)

In [6]:
from ultralytics import YOLO

modelX = YOLO("yolo11n.yaml")
modelX.train(
    data="train_filtered.yaml",
    project="./X_scratch_filtered",
    pretrained=False,  
    epochs = 200,
    patience=10, 
    # save_period=10,
    # time=2,
    # cache="ram",
    batch=-1,
    exist_ok=True
    )

New https://pypi.org/project/ultralytics/8.3.72 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.71  Python-3.11.9 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 3070, 8192MiB)
engine\trainer: task=detect, mode=train, model=yolo11n.yaml, data=train_filtered.yaml, epochs=200, time=None, patience=10, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=./X_scratch_filtered, name=train, exist_ok=True, pretrained=False, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, sh

train: Scanning C:\Users\remote_student\ForeHelm\training\COCO\dataset\train_filtered\labels.cache... 61133 images, 37496 backgrounds, 0 corrupt: 100%|██████████| 98629/98629 [00:00<?, ?it/s]


AutoBatch: Computing optimal batch size for imgsz=640 at 60.0% CUDA memory utilization.
AutoBatch: CUDA:0 (NVIDIA GeForce RTX 3070) 8.00G total, 0.21G reserved, 0.15G allocated, 7.64G free
      Params      GFLOPs  GPU_mem (GB)  forward (ms) backward (ms)                   input                  output
     2591400       6.448         0.610         123.7         141.4        (1, 3, 640, 640)                    list
     2591400        12.9         0.765         19.64         103.8        (2, 3, 640, 640)                    list
     2591400       25.79         1.078         22.08         103.2        (4, 3, 640, 640)                    list
     2591400       51.59         1.741         136.7         105.5        (8, 3, 640, 640)                    list
     2591400       103.2         2.980         46.94         51.67       (16, 3, 640, 640)                    list
AutoBatch: Using batch-size 40 for CUDA:0 4.93G/8.00G (62%) 


train: Scanning C:\Users\remote_student\ForeHelm\training\COCO\dataset\train_filtered\labels.cache... 61133 images, 37496 backgrounds, 0 corrupt: 100%|██████████| 98629/98629 [00:00<?, ?it/s]
val: Scanning C:\Users\remote_student\ForeHelm\training\COCO\dataset\valid_filtered\labels.cache... 7663 images, 4665 backgrounds, 0 corrupt: 100%|██████████| 12328/12328 [00:00<?, ?it/s]


Plotting labels to X_scratch_filtered\train\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.000625), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to X_scratch_filtered\train
Starting training for 200 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

    200/200      6.46G      1.001     0.7866      1.085         81        640: 100%|██████████| 2466/2466 [09:08<00:00,  4.50it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 155/155 [00:33<00:00,  4.66it/s]


                   all      12328      35564      0.775      0.583      0.674      0.481

200 epochs completed in 33.291 hours.
Optimizer stripped from X_scratch_filtered\train\weights\last.pt, 5.5MB
Optimizer stripped from X_scratch_filtered\train\weights\best.pt, 5.5MB

Validating X_scratch_filtered\train\weights\best.pt...
Ultralytics 8.3.71  Python-3.11.9 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 3070, 8192MiB)
YOLO11n summary (fused): 238 layers, 2,583,712 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 155/155 [00:35<00:00,  4.39it/s]


                   all      12328      35564      0.777      0.581      0.674      0.481
                person       6695      27034       0.82      0.653      0.763      0.529
               bicycle        359        691      0.742      0.388      0.486      0.298
                   car       1232       4330      0.711      0.503      0.582      0.369
            motorcycle        346        802      0.806      0.617      0.719      0.467
              airplane        312        596       0.82      0.648      0.733      0.537
                   bus        414        639      0.851      0.651      0.761      0.612
                 train        386        487      0.837      0.791      0.865      0.682
                 truck        632        985      0.627      0.398      0.485      0.351
Speed: 0.1ms preprocess, 0.8ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to X_scratch_filtered\train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 7])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000024034B23350>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047

In [ ]:
from ultralytics import YOLO
model16 = YOLO("X_scratch_filtered/train/weights/last.pt")
model16.train(resume=True)

In [ ]:
from ultralytics import YOLO

model = YOLO("yolo11n.yaml")

model.tune(
    data="train_filtered.yaml",
    project="./hyperparam_tuning2",
    pretrained=False,  
    epochs = 5,
    iterations=300,
    exist_ok=True
    )

Tuner: Initialized Tuner instance with 'tune_dir=hyperparam_tuning2\tune'
Tuner:  Learn about tuning at https://docs.ultralytics.com/guides/hyperparameter-tuning
Tuner: Starting iteration 1/300 with hyperparameters: {'lr0': 0.01, 'lrf': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'warmup_epochs': 3.0, 'warmup_momentum': 0.8, 'box': 7.5, 'cls': 0.5, 'dfl': 1.5, 'hsv_h': 0.015, 'hsv_s': 0.7, 'hsv_v': 0.4, 'degrees': 0.0, 'translate': 0.1, 'scale': 0.5, 'shear': 0.0, 'perspective': 0.0, 'flipud': 0.0, 'fliplr': 0.5, 'bgr': 0.0, 'mosaic': 1.0, 'mixup': 0.0, 'copy_paste': 0.0}


# Validating the YOLO models
The following code is used to validate and compare the different YOLO models that have been trained

This code tries the default and pretrained yolo11n model with the COCO dataset

In [ ]:
from ultralytics import YOLO

# Load a model

model = YOLO("16_scratch_filtered/train/weights/best.pt")
validation_results = model.val(data="train_filtered.yaml", device="0", plots=True)

In [ ]:
# Load a model
model = YOLO("yolo11n.pt")
validation_results = model.val(data="train_filtered.yaml", device="0", plots=True)

In [ ]:
# Validate the model
validation_results.box.map  # map50-95
validation_results.box.map50  # map50
validation_results.box.map75  # map75
validation_results.box.maps  # a list contains map50-95 of each category